<a href="https://colab.research.google.com/github/TayJen/H_Tatarsan2022/blob/main/H_Tatarstan2022_Yolov7x_Multi_data_preparing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В данном ноутбуке помимо использования доступных на соревновании данных будет использован общедоступный датасет с сайта: [CrowdHuman](https://www.crowdhuman.org/)

При обучении модели на столь обильных данных используется свой общий диск, с огромной памятью, вся информация будет сохранена на нем

## Подготовка диска + скачивание файлов

In [1]:
%cd /content/drive/Shareddrives/data_drive
!pwd

/content/drive/Shareddrives/data_drive
/content/drive/Shareddrives/data_drive


In [2]:
!mkdir H_Tatarstan2022
%cd ./H_Tatarstan2022
!pwd

/content/drive/Shareddrives/data_drive/H_Tatarstan2022
/content/drive/Shareddrives/data_drive/H_Tatarstan2022


Открывая данные на сайте можно проверить уникальные айди файлов, для того чтобы скачать их здесь, без ручного скачивания.

Айди файлов:


*   Train Part #1: `134QOvaatwKdy0iIeNqA_p-xkAhkV4F8Y`
*   Train Part #2: `17evzPh7gc1JBNvnW1ENXLy5Kr4Q_Nnla`
*   Train Part #3: `1tdp0UCgxrqy1B6p8LkR-Iy0aIJ8l4fJW`
*   Annotation Train: `1UUTea5mYqvlUObsC1Z8CFldHJAtLtMX3`



Вспомогательные ссылки:


*   [Downloading data from a shared google drive link in google colab](https://stackoverflow.com/questions/62759748/downloading-data-from-a-shared-google-drive-link-in-google-colab)
*   [Gdown official documentation](https://github.com/wkentaro/gdown)

In [5]:
import gdown

ids = ['134QOvaatwKdy0iIeNqA_p-xkAhkV4F8Y',
       '17evzPh7gc1JBNvnW1ENXLy5Kr4Q_Nnla',
       '1tdp0UCgxrqy1B6p8LkR-Iy0aIJ8l4fJW',
       '1UUTea5mYqvlUObsC1Z8CFldHJAtLtMX3']

for id in ids:
    gdown.download(id=id, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=134QOvaatwKdy0iIeNqA_p-xkAhkV4F8Y
To: /content/drive/Shareddrives/data_drive/H_Tatarstan2022/CrowdHuman_train01.zip
100%|██████████| 2.97G/2.97G [00:37<00:00, 79.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=17evzPh7gc1JBNvnW1ENXLy5Kr4Q_Nnla
To: /content/drive/Shareddrives/data_drive/H_Tatarstan2022/CrowdHuman_train02.zip
100%|██████████| 3.09G/3.09G [00:57<00:00, 53.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1tdp0UCgxrqy1B6p8LkR-Iy0aIJ8l4fJW
To: /content/drive/Shareddrives/data_drive/H_Tatarstan2022/CrowdHuman_train03.zip
100%|██████████| 2.31G/2.31G [00:34<00:00, 67.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1UUTea5mYqvlUObsC1Z8CFldHJAtLtMX3
To: /content/drive/Shareddrives/data_drive/H_Tatarstan2022/annotation_train.odgt
100%|██████████| 80.0M/80.0M [00:02<00:00, 31.8MB/s]


In [6]:
!ls

annotation_train.odgt	CrowdHuman_train02.zip
CrowdHuman_train01.zip	CrowdHuman_train03.zip


In [7]:
!mkdir ./add_data
!mkdir ./add_data/zipped
!mkdir ./add_data/unzipped
!mkdir ./add_data/annotations
!mv CrowdHuman_train01.zip ./add_data/zipped
!mv CrowdHuman_train02.zip ./add_data/zipped
!mv CrowdHuman_train03.zip ./add_data/zipped
!mv annotation_train.odgt ./add_data/annotations

In [ ]:
!unzip ./add_data/zipped/CrowdHuman_train01.zip -d ./add_data/unzipped
!unzip ./add_data/zipped/CrowdHuman_train02.zip -d ./add_data/unzipped
!unzip ./add_data/zipped/CrowdHuman_train03.zip -d ./add_data/unzipped

## Конвертация аннотаций в Yolo формат

Далее нам необходимо трансформировать используемые аннотации в Yolo формат, так как обучаться будет state-of-the-art Yolov7x

Вспомогательные ссылочки:


*   [Convert CrowdHuman dataset to Yolo annotations](https://gist.github.com/adujardin/62653118466962264aa0c6339c3e9cf5)
*   [crowdhuman_train_anno.py](https://github.com/alaksana96/darknet-crowdhuman/blob/master/crowdhuman_train_anno.py)

In [9]:
!mkdir ./add_data/unzipped/Yolo_labels

In [11]:
import numpy
import cv2
from multiprocessing import Process, Manager
import yaml
from tqdm import tqdm

intPERSON = 0
intHEAD   = 1

manager = Manager()
return_dict = manager.dict()
return_dict

<DictProxy object, typeid 'dict' at 0x7f7d08f19210>

In [12]:
def generate_annotations(ii, return_dict):
    line = return_dict[ii]
    del return_dict[ii]
    
    dictLine = yaml.load(line)

    strID = dictLine['ID']

    img = cv2.imread('./add_data/unzipped/Images/{}.jpg'.format(strID), 1)

    imgWidth  = img.shape[1]
    imgHeight = img.shape[0]

    # Create .txt label file
    with open('./add_data/unzipped/Yolo_labels/{}.txt'.format(strID), 'w+') as txtf:

        for label in dictLine['gtboxes']:

            if 'extra' in label and 'ignore' in label['extra'] and label['extra']['ignore'] == 1:
                continue
            if 'extra' in label and 'unsure' in label['extra'] and label['extra']['unsure'] == 1:
                continue 

            # Person BB
            px = float(label['fbox'][0])
            py = float(label['fbox'][1])
            pw = float(label['fbox'][2])
            ph = float(label['fbox'][3])

            # Head BB
            hx = float(label['hbox'][0])
            hy = float(label['hbox'][1])
            hw = float(label['hbox'][2])
            hh = float(label['hbox'][3])

            # Absolute person BB
            cpx = px + pw / 2
            cpy = py + ph / 2

            abspx = cpx / imgWidth
            abspy = cpy / imgHeight
            abspw = pw / imgWidth
            absph = ph / imgHeight  

            abspx = 1 if abspx > 1 else abspx
            abspy = 1 if abspy > 1 else abspy
            abspw = 1 if abspw > 1 else abspw
            absph = 1 if absph > 1 else absph
            abspx = 0.000001 if abspx < 0 else abspx
            abspy = 0.000001 if abspy < 0 else abspy
            abspw = 0.000001 if abspw < 0 else abspw
            absph = 0.000001 if absph < 0 else absph

            # Absolute head BB
            chx = hx + hw / 2
            chy = hy + hh / 2

            abshx = chx / imgWidth
            abshy = chy / imgHeight
            abshw = hw / imgWidth
            abshh = hh / imgHeight  

            abshx = 1 if abshx > 1 else abshx
            abshy = 1 if abshy > 1 else abshy
            abshw = 1 if abshw > 1 else abshw
            abshh = 1 if abshh > 1 else abshh
            abshx = 0.000001 if abshx < 0 else abshx
            abshy = 0.000001 if abshy < 0 else abshy
            abshw = 0.000001 if abshw < 0 else abshw
            abshh = 0.000001 if abshh < 0 else abshh

            # Write to file
            txtf.write('{} {:.6f} {:.6f} {:.6f} {:.6f}\n'.format(intPERSON,
                                                                 abspx,
                                                                 abspy,
                                                                 abspw,
                                                                 absph))

            txtf.write('{} {:.6f} {:.6f} {:.6f} {:.6f}\n'.format(intHEAD,
                                                                 abshx,
                                                                 abshy,
                                                                 abshw,
                                                                 abshh))

In [13]:
with open('./add_data/annotations/annotation_train.odgt') as f:
        
    processes = []
    max_iter = 500

    for ii, line in tqdm(enumerate(f)): 
        return_dict[ii] = line
        pcs = Process(target=generate_annotations,
                      args=(ii, return_dict))
        processes.append(pcs)
        pcs.start()
        
        if ii % max_iter == 0:
            for jj in range(len(processes)):    
                processes[jj].join()  

            processes = []
            
    for jj in range(len(processes)):    
        processes[jj].join()      
        
    processes = []

15000it [32:52,  7.60it/s]


In [15]:
import os

len(os.listdir('./add_data/unzipped/Yolo_labels'))

15000

In [16]:
len(os.listdir('./add_data/unzipped/Images'))

15000

## Перебрасывание старых данных на общий диск

Перебросим данные с личного диска, куда они были скачаны в самом начале в новый диск, для последующего объединения с имеющимися новыми данными

In [17]:
!mkdir ./data
!mkdir ./data/zipped
!cp /content/drive/MyDrive/H_Tatarstan2022/data/zipped/train_dataset_train.zip ./data/zipped/train_dataset_train.zip
!cp /content/drive/MyDrive/H_Tatarstan2022/data/zipped/test_dataset_test.zip ./data/zipped/test_dataset_test.zip

In [ ]:
!unzip ./data/zipped/train_dataset_train.zip -d ./data/
!unzip ./data/zipped/test_dataset_test.zip -d ./data/

In [19]:
import os

len(os.listdir('./data/train/images')), len(os.listdir('./data/test/images'))

(2002, 855)

In [20]:
import pandas as pd

pd.Series([file_name.split('_')[-1]
           for file_name in os.listdir('data/train/labels')]).value_counts()

head..txt        1741
human..txt       1740
car..txt         1647
face..txt         630
carplate..txt     246
dtype: int64

## Yolo разметка для старых данных

Будет создано три различные директории с данными, одна для людей/голов, одна для лиц и одна для машин/госномеров

Прежде чем мы создадим директории, трансформируем аннотации в нужный формат

In [25]:
# Сделаем разметку как в Yolo, отдельно для людей и отдельно для машин
# После этого нужным образом перемешаем и закинем в папку data_for_yolo

name_to_class_num_human = {
    'human': 0,
    'head': 1
}

name_to_class_num_face = {
    'face': 0
}

name_to_class_num_car = {
    'car': 0,
    'carplate': 1
}

In [26]:
# !mkdir ./data/train/yolo_labels_human
# !mkdir ./data/train/yolo_labels_car
!mkdir ./data/train/yolo_labels_face

In [27]:
def yolo_annotation_new():
    human_set = {'human', 'head'}
    face_set = {'face'}
    car_set = {'car', 'carplate'}

    old_path = './data/train/labels/'
    new_path_human = './data/train/yolo_labels_human/'
    new_path_face = './data/train/yolo_labels_face/'
    new_path_car = './data/train/yolo_labels_car/'

    for img_name in tqdm(os.listdir('./data/train/images/')):
        img_name = img_name.split('.')[0]
        for f in os.listdir(old_path):
            if f.startswith(img_name):
                class_name = f.split('.')[0].split('_')[-1]
                if class_name in human_set:
                    class_num = name_to_class_num_human[class_name]
                    # Так как файл будет открываться несколько раз
                    # То нам необходимо добавлять информацию в файл,
                    # А не перезаписывать его
                    with open(new_path_human + img_name + '.txt', 'a+') as f_new, open(old_path + f, 'r') as f_old:
                        for row in f_old.readlines():
                            f_new.write(str(class_num) + row[1:])
                
                elif class_name in face_set:
                    class_num = name_to_class_num_face[class_name]
                    with open(new_path_face + img_name + '.txt', 'a+') as f_new, open(old_path + f, 'r') as f_old:
                        for row in f_old.readlines():
                            f_new.write(str(class_num) + row[1:])
                
                elif class_name in car_set:
                    class_num = name_to_class_num_car[class_name]
                    with open(new_path_car + img_name + '.txt', 'a+') as f_new, open(old_path + f, 'r') as f_old:
                        for row in f_old.readlines():
                            f_new.write(str(class_num) + row[1:])

yolo_annotation_new()

100%|██████████| 2002/2002 [40:22<00:00,  1.21s/it]


In [28]:
len(os.listdir('./data/train/yolo_labels_human/'))

1741

In [29]:
len(os.listdir('./data/train/yolo_labels_face/'))

630

In [30]:
len(os.listdir('./data/train/yolo_labels_car/'))

1893

## Человек/Голова

В данном разделе помимо перебрасывания старых картинок нам также необходимо учесть новые. В используемом подходе все новые картинки будут использоваться для обучения, однако не будут использоваться для валидации, так как тестовый набор на сайте не будет их содержать, а валидация обязана имитировать тестовое распределение

In [31]:
!mkdir ./data_for_yolo_human
!mkdir ./data_for_yolo_human/data
!mkdir ./data_for_yolo_human/data/images
!mkdir ./data_for_yolo_human/data/labels
!mkdir ./data_for_yolo_human/data/images/train
!mkdir ./data_for_yolo_human/data/labels/train
!mkdir ./data_for_yolo_human/data/images/test
!mkdir ./data_for_yolo_human/data/labels/test

In [36]:
import yaml


def create_yaml_cont_human():
    """
        train: /content/drive/Shareddrives/data_drive/H_Tatarstan2022/data_for_yolo_human/data/images/train/
        val: /content/drive/Shareddrives/data_drive/H_Tatarstan2022/data_for_yolo_human/data/images/test/

        # number of classes
        nc: 2

        # class names
        names: ['human', 'head']
    """

    yaml_content = {
        'train': '/content/drive/Shareddrives/data_drive/H_Tatarstan2022/data_for_yolo_human/data/images/train/',
        'val': '/content/drive/Shareddrives/data_drive/H_Tatarstan2022/data_for_yolo_human/data/images/test/',
        'nc': 2,
        'names': ['human', 'head']
    }

    with open('./data_for_yolo_human/dataset.yaml', 'w') as f_yml:
        yaml.dump(yaml_content, f_yml)

create_yaml_cont_human()

### Yolo transfer + shuffle

In [37]:
labels_names = os.listdir('./data/train/yolo_labels_human')
print(len(labels_names))

1741


In [38]:
from sklearn.model_selection import train_test_split


train_labels, test_labels = train_test_split(labels_names, test_size=0.25,
                                             shuffle=True, random_state=59)

len(train_labels), len(test_labels)

(1305, 436)

In [39]:
train_labels = set(train_labels)
test_labels = set(test_labels)

len(train_labels), len(test_labels)

(1305, 436)

In [40]:
from shutil import copy


def copy_to_yolo_human():
    old_path_img = './data/train/images/'
    old_path_lbl = './data/train/yolo_labels_human/'
    yolo_path_img = './data_for_yolo_human/data/images/'
    yolo_path_lbl = './data_for_yolo_human/data/labels/'

    for lbl_name in tqdm(os.listdir(old_path_lbl)):
        image_name = lbl_name.split('.')[0] + '.jpg'
        # print(image_name, lbl_name)
        if lbl_name in train_labels:
            copy(old_path_img + image_name, yolo_path_img + 'train/' + image_name)
            copy(old_path_lbl + lbl_name, yolo_path_lbl + 'train/' + lbl_name)
        elif lbl_name in test_labels:
            copy(old_path_img + image_name, yolo_path_img + 'test/' + image_name)
            copy(old_path_lbl + lbl_name, yolo_path_lbl + 'test/' + lbl_name)

copy_to_yolo_human()

100%|██████████| 1741/1741 [00:53<00:00, 32.65it/s]


In [41]:
len(os.listdir('./data_for_yolo_human/data/images/train')), len(os.listdir('./data_for_yolo_human/data/images/test'))

(1305, 436)

In [42]:
len(os.listdir('./data_for_yolo_human/data/labels/train')), len(os.listdir('./data_for_yolo_human/data/labels/test'))

(1305, 436)

Так же перебросим новые данные в `train`

In [44]:
def copy_to_yolo_human_new():
    old_path_img = './add_data/unzipped/Images/'
    old_path_lbl = './add_data/unzipped/Yolo_labels/'
    yolo_path_img = './data_for_yolo_human/data/images/train/'
    yolo_path_lbl = './data_for_yolo_human/data/labels/train/'

    for lbl_name in tqdm(os.listdir(old_path_lbl)):
        image_name = lbl_name.split('.')[0] + '.jpg'
        copy(old_path_img + image_name, yolo_path_img + image_name)
        copy(old_path_lbl + lbl_name, yolo_path_lbl + lbl_name)

copy_to_yolo_human_new()

100%|██████████| 15000/15000 [1:14:28<00:00,  3.36it/s]


In [45]:
len(os.listdir('./data_for_yolo_human/data/images/train')), len(os.listdir('./data_for_yolo_human/data/images/test'))

(16305, 436)

In [46]:
len(os.listdir('./data_for_yolo_human/data/labels/train')), len(os.listdir('./data_for_yolo_human/data/labels/test'))

(16305, 436)

## Лицо

Так как во всех данных лицо заблюрено, то не имеет смысла искать похожие датасеты

In [47]:
!mkdir ./data_for_yolo_face
!mkdir ./data_for_yolo_face/data
!mkdir ./data_for_yolo_face/data/images
!mkdir ./data_for_yolo_face/data/labels
!mkdir ./data_for_yolo_face/data/images/train
!mkdir ./data_for_yolo_face/data/labels/train
!mkdir ./data_for_yolo_face/data/images/test
!mkdir ./data_for_yolo_face/data/labels/test

In [48]:
import yaml


def create_yaml_cont_face():
    """
        train: /content/drive/Shareddrives/data_drive/H_Tatarstan2022/data_for_yolo_face/data/images/train/
        val: /content/drive/Shareddrives/data_drive/H_Tatarstan2022/data_for_yolo_face/data/images/test/

        # number of classes
        nc: 1

        # class names
        names: ['face']
    """

    yaml_content = {
        'train': '/content/drive/Shareddrives/data_drive/H_Tatarstan2022/data_for_yolo_face/data/images/train/',
        'val': '/content/drive/Shareddrives/data_drive/H_Tatarstan2022/data_for_yolo_face/data/images/test/',
        'nc': 1,
        'names': ['face']
    }

    with open('./data_for_yolo_face/dataset.yaml', 'w') as f_yml:
        yaml.dump(yaml_content, f_yml)

create_yaml_cont_face()

### Yolo transfer + shuffle

In [49]:
labels_names = os.listdir('./data/train/yolo_labels_face')
len(labels_names)

630

In [50]:
from sklearn.model_selection import train_test_split


train_labels, test_labels = train_test_split(labels_names, test_size=0.25,
                                             shuffle=True, random_state=59)

len(train_labels), len(test_labels)

(472, 158)

In [51]:
train_labels = set(train_labels)
test_labels = set(test_labels)

len(train_labels), len(test_labels)

(472, 158)

In [52]:
def copy_to_yolo_face():
    old_path_img = './data/train/images/'
    old_path_lbl = './data/train/yolo_labels_face/'
    yolo_path_img = './data_for_yolo_face/data/images/'
    yolo_path_lbl = './data_for_yolo_face/data/labels/'

    for lbl_name in tqdm(os.listdir(old_path_lbl)):
        image_name = lbl_name.split('.')[0] + '.jpg'
        if lbl_name in train_labels:
            copy(old_path_img + image_name, yolo_path_img + 'train/' + image_name)
            copy(old_path_lbl + lbl_name, yolo_path_lbl + 'train/' + lbl_name)
        elif lbl_name in test_labels:
            copy(old_path_img + image_name, yolo_path_img + 'test/' + image_name)
            copy(old_path_lbl + lbl_name, yolo_path_lbl + 'test/' + lbl_name)

copy_to_yolo_face()

100%|██████████| 630/630 [03:39<00:00,  2.86it/s]


In [53]:
len(os.listdir('./data_for_yolo_face/data/images/train')), len(os.listdir('./data_for_yolo_face/data/images/test'))

(472, 158)

In [54]:
len(os.listdir('./data_for_yolo_face/data/labels/train')), len(os.listdir('./data_for_yolo_face/data/labels/test'))

(472, 158)

## Машина/Госномер

Прошлые модели показывали отличный скор на машинах и их госномерах, дополнительные данные не требуются

In [55]:
!mkdir ./data_for_yolo_car
!mkdir ./data_for_yolo_car/data
!mkdir ./data_for_yolo_car/data/images
!mkdir ./data_for_yolo_car/data/labels
!mkdir ./data_for_yolo_car/data/images/train
!mkdir ./data_for_yolo_car/data/labels/train
!mkdir ./data_for_yolo_car/data/images/test
!mkdir ./data_for_yolo_car/data/labels/test

In [56]:
def create_yaml_cont_car():
    """
        train: /content/drive/Shareddrives/data_drive/H_Tatarstan2022/data_for_yolo_car/data/images/train/
        val: /content/drive/Shareddrives/data_drive/H_Tatarstan2022/data_for_yolo_car/data/images/test/

        # number of classes
        nc: 2

        # class names
        names: ['car', 'carplate']
    """

    yaml_content = {
        'train': '/content/drive/Shareddrives/data_drive/H_Tatarstan2022/data_for_yolo_car/data/images/train/',
        'val': '/content/drive/Shareddrives/data_drive/H_Tatarstan2022/data_for_yolo_car/data/images/test/',
        'nc': 2,
        'names': ['car', 'carplate']
    }

    with open('./data_for_yolo_car/dataset.yaml', 'w') as f_yml:
        yaml.dump(yaml_content, f_yml)

create_yaml_cont_car()

### Yolo transfer + shuffle

In [57]:
labels_names = os.listdir('./data/train/yolo_labels_car')
len(labels_names)

1893

In [58]:
train_labels, test_labels = train_test_split(labels_names, test_size=0.25,
                                             shuffle=True, random_state=59)

len(train_labels), len(test_labels)

(1419, 474)

In [59]:
train_labels = set(train_labels)
test_labels = set(test_labels)

len(train_labels), len(test_labels)

(1419, 474)

In [ ]:
def copy_to_yolo_car():
    old_path_img = './data/train/images/'
    old_path_lbl = './data/train/yolo_labels_car/'
    yolo_path_img = './data_for_yolo_car/data/images/'
    yolo_path_lbl = './data_for_yolo_car/data/labels/'

    for lbl_name in tqdm(os.listdir(old_path_lbl)):
        image_name = lbl_name.split('.')[0] + '.jpg'
        if lbl_name in train_labels:
            copy(old_path_img + image_name, yolo_path_img + 'train/' + image_name)
            copy(old_path_lbl + lbl_name, yolo_path_lbl + 'train/' + lbl_name)
        elif lbl_name in test_labels:
            copy(old_path_img + image_name, yolo_path_img + 'test/' + image_name)
            copy(old_path_lbl + lbl_name, yolo_path_lbl + 'test/' + lbl_name)

copy_to_yolo_car()

 54%|█████▍    | 1025/1893 [04:43<03:53,  3.72it/s]

In [ ]:
len(os.listdir('./data_for_yolo_car/data/images/train')), len(os.listdir('./data_for_yolo_car/data/images/test'))

In [ ]:
len(os.listdir('./data_for_yolo_car/data/labels/train')), len(os.listdir('./data_for_yolo_car/data/labels/test'))